# Quantitative Biology Lab - Week 1
## Radhika Jangi

## 1. Coverage Analysis

### a. Genome length

> samtools faidx ref.fa<br/>
> cat ref.fa.fai
   
The genome is 233806 bases

### b. Number/length of reads
> fastqc

Open each .fq file in FastQC

frag180.1: 35178 sequences, 100 bp in length<br/>
frag180.2: 35178 sequences, 100 bp in length<br/>
jump2k.1: 70355 sequences, 50 bp in length<br/>
jump2k.2: 70355 sequences, 50 bp in length

### c. Coverage

180.1: (35178 * 100)/233806 ~ 15X <br/>
180.2: (35178 * 100)/233806 ~ 15X<br/>
2k.1: (70355 * 50)/233806 ~ 15X<br/>
2k.2: (70355 * 50)/233806 ~ 15X

Total coverage ~ 60X

### d. Average Quality Value Plot<br/>
Plot images are in assignment1 directory

## 2. Kmer Analysis

#### a. Histogram<br/>
> jellyfish count -m 21 -C -s 1000000 /Users/cmdb/qbb2020-answers/assignment1/*.fq <br/>
> jellyfish histo mer_counts.jf > reads.histo <br/>

Exactly 1091 kmers show up in the 50 bin

#### b. Top 10 Kmers<br/>
> jellyfish dump -c -t mer_counts.jf > mer_counts_dumps.fa<br/>

> sort -k 2 -r -n  mer_counts_dumps.fa | head -n 10

GCCCACTAATTAGTGGGCGCC	105<br/>
CGCCCACTAATTAGTGGGCGC	104<br/>
CCCACTAATTAGTGGGCGCCG	104<br/>
ACGGCGCCCACTAATTAGTGG	101<br/>
CAGGCCAGCTTATAAGCTGGC	98<br/>
AACAGGCCAGCTTATAAGCTG	98<br/>
ACAGGCCAGCTTATAAGCTGG	97<br/>
AGGCCAGCTTATAAGCTGGCC	95<br/>
AGCATCGCCCACATGTGGGCG	83<br/>
GCATCGCCCACATGTGGGCGA	82

#### c. Estimated Genome Size<br/>
Genome Haploid Length<br/>        
Min: 233,468 bp   Max: 233,805 bp

#### d. Size Estimate<br/>
The sizes are very close, but GenomeScope's max estimate is 1 base under the value reported by Samtools. The software's calculation is based off of the histogram versus Samtools, which counts each base in the reference file itself.

## 3. De novo Assembly

#### a. Contig number
> spades.py --pe1-1 frag180.1.fq --pe1-2 frag180.2.fq --mp1-1 jump2k.1.fq --mp1-2 jump2k.2.fq -o asm -t 4 -k 31<br/>
> grep -c '>' contigs.fasta<br/>

Four contigs were produced

#### b. Total contig length
> samtools faidx contigs.fasta | sort -n

In [1]:
f = open('/Users/cmdb/qbb2020-answers/assignment1/asm/contigs.fasta.fai', 'r')
contigs = f.readlines()
f.close()
contig_len = 0
for line in contigs:
    strip_line = line.strip().split("\t")
    #print(strip_line)
    next_con = int(strip_line[1])
    contig_len+=next_con
print("Total contig length is {}".format(contig_len))

Total contig length is 234467


#### c. Contig Length
> samtools faidx contigs.fasta | sort -n
> less -S contigs.fasta.fai

Node 1 is the longest at 105831 bases

#### d. Contig N50 size
Write a script to sum contigs until half of total contig sum is covered

In [2]:
f = open('/Users/cmdb/qbb2020-answers/assignment1/asm/contigs.fasta.fai', 'r')
contigs = f.readlines()
f.close()

half_total_contig = 234467/2
n50 = 0
node_count = 0
while n50 < half_total_contig:
    strip_line = contigs[node_count].strip().split("\t")
    next_con_len = int(strip_line[1])
    n50+= next_con_len
    node_count+=1
print("The N50 is {}".format(strip_line[1]))

The N50 is 47861


## 4. Whole Genome Alignment

#### a. Average Identity
> dnadiff /Users/cmdb/qbb2020-answers/assignment1/ref.fa /Users/cmdb/qbb2020-answers/assignment1/asm/contigs.fasta<br/>

The alignments average identity is 100%. This makes sense because the total coverage is 60X

#### b. Longest alignment
> mkdir nucmerdir, cd nucmerdir<br/>
> nucmer /Users/cmdb/qbb2020-answers/assignment1/ref.fa /Users/cmdb/qbb2020-answers/assignment1/asm/contigs.fasta<br/>
> show-coords out.delta<br/>

105831 bases is the longest alignment length.

#### c. Insertions and Deletions
> cd ..<br/>
> cat out.report<br/>

There are 5 deletions and 1 insertion in the query.

## 5. Decoding the Insertion

#### a. Where's the insertion?
Looking at show-coords output, we can see a missing region in Node 3. 13854 - 14565 is the insertion

#### b. How long?
712 bp

#### c. Sequence extraction
> samtools faidx /Users/cmdb/qbb2020-answers/assignment1/asm/contigs.fasta NODE_3_length_41352_cov_20.588756:13854-14565<br/>

\>NODE_3_length_41352_cov_20.588756:13854-14565
TAACGATTTACATCGGGAAAGCTTAATGCAATTCACGCAGATATTCAGCTTAGAAGGTAC
GCAGCGGTGACGGGGTGCGGTCCATAATCTATGAAGCTATGAATTCGTACCTCAAGTAAT
GTTTTCTTCGCTGCAGTTCAGAAGTGATAAAGGTATCCCGCTTAGCCTGGCATACTTTGT
GCGTTCGTACCGCCCAGCATTAATGACTTGTGTAGGCAAGTAATGAACGACTCTTCTACG
CCGCGCCTAACCTCCGCACATAATGGCAGCATGTGGTAGTTACATACGCACAGAAGTGGT
TCGGTTTTAACTATAGTCAGATATGAATAAGCTGCGTGTGTCGTTGTGTCGGCGTGTCGT
ACTTACCTCCTGACATAGGTGAATTTCAGCCTACTGTAAGTTTGGAGTCGCGCTCTTTTC
TTATTATATTCTTTGGTATGTGTGTGATGGGTTCGGGCGTGTATTGATGTCTCTAAGGCT
CATGTTAGTGTTTATTTGGTCAGTTATGACGGTGTTCCTGTCGTACGTGTTGGCTTAGCG
GACTTGTAGACGGGATCAAGGTTGTCTGACCCTCCGGTCGACCGTGGGTCGGCCGTCCCG
GCCAGAATACAAGCCGCTTAGACTTTCGAAAGAGGGTAAGTTACTACGCGCGAACGTTAT
ACCTCGTTTCAGTATGCACTCCCTTAAGTCACTCAGAAAAGACTAAGGGGCT


#### d. Decoded message
> chmod +x ported_decoder.py<br/>
> ./ported_decoder.py -d --rev_comp --input insertion.txt

The decoded message :  Congratulations to the 2020 CMDB @ JHU class!  Keep on looking for little green aliens...